***
# 深層学習day4
***

## Section1: 強化学習
+ 教師あり学習・教師なし学習とは別の枠組み
+ 行動の結果として得られる利益が最大化されるように行動を改善していく
+ 答えがある訳ではなく，試行錯誤から答えを導き出す
+ キーワード
  + エージェント：行動を起こす主体
  + 環境：エージェントが行動する対象
  + 行動：エージェントが行う行為
  + 状態：エージェントが置かれている状況
  + 方策：エージェントが行う行為の実施ルール
  + 報酬：行動に対する即時的な結果
  + 収益：未来方向の報酬の和
  + 価値：エージェントが行った行動の得られる収益の期待値
+ 探索と利用のトレードオフ
  + 試行錯誤（ランダム）な行動だけでは学習が進まない
  + 学習結果のみで行動すると新たな発見が生まれない
+ 強化学習の学習：優れた方策を見つけることが目標．計算量が問題だった
  + Q学習：行動するたびに更新
  + 関数近似法：価値観数や方策関数を関数近似⇒ニューラルネットワークを使う
    + 方策：方策関数π(s, a)  
      状態と行動をパラメータとして行動の確率を求める
    + 価値：  
      状態価値関数V(s)：状態をパラメータとして価値を求める  
      行動価値関数Q(s, a)：状態と行動をパラメータとして価値を求める
+ 方策勾配法
$$
\pi(s, a | \theta)
$$
について，
$$
\theta^{t+1} = \theta^{t} + \epsilon \nabla_{\theta} J(\theta) \\
\nabla_{\theta} J(\theta) = \nabla_{\theta}\sum_{a \in A} \pi_{\theta}(a|s)Q^{\pi}(s, a) =
E_{\pi}\left[ Q^{\pi}(s, a)\nabla_{\theta} \log \pi_{\theta}(a|s)\right ]
$$
現在の状態で複数の行動が選択できる．それぞれの行動について価値が計算できる．全ての行動パターンについての価値の総和を計算している



【参考文献】
1. 牧野ら編著『これからの強化学習』森北出版 2016.10
2. 曽我部東馬著『強化学習アルゴリズム入門』オーム社 2019.05

## Section2: AlphaGo
+ Alpha Go Lee
  + 畳み込みニューラルネットワークで方策関数と価値関数を学習
  + 方策関数：PolicyNet
    + 盤面特徴入力(H:19,W:19,C:48)
    + 畳み込み層(K:5×5,stride:1, zeropad:2, F:192)
    + 活性化関数(ReLU)
    + (畳み込み層+ReLU)×11　(K:3×3,stride:1, zeropad:1, F:192)
    + 畳み込み層(K:1×1,stride:1,F:1)
    + 活性化関数(SoftMax)
    + Policy出力(19×19)
  + 価値関数：ValueNet
    + 盤面特徴入力(H:19,W:19,C:49)手番がどちらかを示すチャネルが追加
    + 畳み込み層(K:5×5,stride:1, zeropad:2, F:192)
    + 活性化関数(ReLU)
    + (畳み込み層+ReLU)×11　(K:3×3,stride:1, zeropad:1, F:192)
    + 畳み込み層(K:1×1,stride:1,F:1)
    + 全結合(256)←ここで1次元配列に
    + 全結合(1)
    + 活性化関数(tanh)
    + Value出力(-1～1)
  + 学習のステップ
    + PolicyNet同士で対局シミュレーションを実施し，その結果を用いて方策勾配法でパラメータを更新
    + ValueNetでは，対局シミュレーションの勝敗を教師データとして学習を進める．  
      対局シミュレーションは途中までを作成済みのPolicyNetを使い，途中から強化学習で生成したPolicyNetで終局まで進め，その勝敗を報酬とする
    + RollOutPolicy  
      PolicyNetよりも早い計算処理できる線形の方策関数．
    + モンテカルロ木探索：価値関数
    1. 人間対人間の棋譜を教師データとしてPolicyNetとRollOutPolicyで学習する
    2. 強化学習でPolicyNetを学習
    3. 強化学習でValueNetを学習
  + モンテカルロ木探索の詳細  
    現局面をルートノードとして制限時間内に探索を繰り返し行う．最も多く探索された経路を選択する．
    + Selection：リーフノードに至るまでノード選択を繰り返す．選択の基準は，行動価値関数と探索初期にランダム生成をもたせる項との和が最大になるもの．
    + Expansion：リーフノードの到達回数が設定回数を超えた場合，新たなノードを追加する．新たなノードの選択確率はRollOutPolicyで計算される．
    + Evaluation：リーフノードのValueNet値が未評価であれば評価する．
    + Backup：ValueNetの評価後に探索回数，評価回数，探索時勝利回数，評価時勝利回数，行動価値関数の更新を行う



+ Alpha Go Zero
  + 教師あり学習を使わず，強化学習のみ
  + 特徴入力からヒューリスティックな(人の判断による)要素を排除
  + PolicyNetとValueNetを統合
  + ResidualNetの導入
  + モンテカルロ木探索でのRollOutシュミレーションを削除  
  <br>  
  + PolicyValueNet
    + 盤面特徴入力(H:19,W:19,C:17)
    + 畳み込み層(K:3×3,stride:1,F:256)
    + バッチ正規化
    + 活性化関数(ReLU)
    + ResidualBlock×39（⇒アンサンブル効果がある？）
      + 畳み込み層(K:3×3,stride:1,F:256)
      + バッチ正規化
      + 活性化関数(ReLU)
      + 畳み込み層(K:3×3,stride:1,F:256)
      + バッチ正規化
      + 入力元データのショートカット結合(勾配消失爆発問題対策)
      + 活性化関数(ReLU)
    + ネットワークの分岐
      1. 方策関数用
        + 畳み込み層(K:1×1,stride:1,F:2)
        + バッチ正規化
        + 活性化関数(ReLU)
        + 全結合(362)
        + 活性化関数(SoftMax)
        + Policy出力
      2. 価値関数用
        + 畳み込み層(K:1×1,stride:1,F:1)
        + バッチ正規化
        + 活性化関数(ReLU)
        + 全結合(256)
        + 活性化関数(ReLU)
        + 全結合(1)
        + 活性化関数(tanh)
        + Value出力(-1～1)
<br>  
  + 学習手法
    1. 自己対局
      + モンテカルロ木探索を用いる
      + 30手目まではランダム
      + 局面ごとのシミュレーション回数，勝敗を記録
    2. 学習
      + PolicyValueNetによる学習
      + Policy部分ではクロスエントロピー誤差，Value部分では平均二乗誤差を利用
    3. ネットワーク更新
      + 現状のネットワークと学習後のネットワークで対局を実施
      + 勝率が高い方のネットワークで更新

【参考文献】
1. 牧野ら編著『これからの強化学習』森北出版 2016.10
2. 曽我部東馬著『強化学習アルゴリズム入門』オーム社 2019.05
3. 久保隆宏著『Pythonで学ぶ強化学習　改訂第2版』講談社 2019.09

## Section3: 軽量化・高速化技術
+ 分散深層学習  
  モデルが複雑に，データ量の増加で計算量が年10倍増加しているがコンピュータの性能は18ヶ月で2倍⇒複数のコンピュータで同時に処理する
  + ワーカー：学習させる装置の単位．ワーカーは個別のコンピュータというわけではなく，GPUを1ワーカーとしてもよい．スマートフォンもワーカーとなり得る．
  + データ並列化：学習モデルを複製し，ワーカーを複数用意．データを分割してそれぞれのワーカーに計算させる．
    + 同期型：各ワーカーの演算を集計し，平均値で更新．学習スピードは早いが処理スピードは遅い．
    + 非同期型：演算の結果は同期をとらず随時更新，各ワーカーからの演算結果を次の演算のパラメータとして利用．処理スピードは早いが学習スピードは遅い
  + モデル並列：モデルを複数のパーツに分割して各ワーカーで処理．ネットワーク分岐のあるモデルと相性が良い．各分割モデルを各GPUに配置することが多い．パラメータ数が多いほど効率が上がる．
+ GPU高速化  
  低性能だが単純な並列処理が得意なコアが多数．本来は画像処理に利用⇒ニューラルネットワークと相性が良い
  + GPGPU(General Purpose on GPU)
    + CUDA: NVIDIA用．Deep Learning用に提供されている
    + OpenCL: 他社GPUでも使用可能．Deep Learningに特化はしていない．  

    ※GPGPUを意識せずにTensorflowやPyTorch内で利用できる
+ 軽量化
  + 量子化(Quantization)  
    ネットワークが大きくなるとパラメータが多いので演算量とメモリ消費量が大きくなるので，数値の精度を落とすことで演算量とメモリ消費量を削減（16bitにしてもネットワークの精度は大きく変わらない）
+ 蒸留(Knowlegde Distillation)  
  学習済みの精度の高いモデルから軽量モデルへ知識を継承
  + 教師モデル：大きい，複雑，精度が高い
  + 生徒モデル：軽い，単純，精度が落ちる？⇒若干落ちる？  

  学習済み教師モデルからの出力(固定)と生徒モデルからの出力を結合して生徒モデルの重みを更新
+ プルーニング  
  大きなネットワークの中で精度に寄与していないニューロンを削除することでモデルの軽量化，高速化

  + 重みが閾値以下のニューロンを削除（削減数の割に精度は下がらない）
  + ドロップアウトは過学習を防ぐため，重み更新の度に一部のニューロンをランダムに不活性化させる


## Section4: 応用技術
2017年頃に画像認識の技術は精度面で完成されており，今は計算量の削減方法を改善している
+ 一般的な畳み込み層
  + 入力層：(H, W, C)
  + 畳み込み層：(K, K, C)×F
  + 出力層：(H, W, F)
  + 畳み込み演算の計算量：H×W×K×K×C×F
+ MobileNets  
  Depthwise ConvolutionとPointwise Convolutionで計算量を削減
  + Depthwise Convolution
    + 畳み込み層のカーネルは(K, K, 1)×1で各チャネルを1枚のフィルタで処理
    + 畳み込み演算の計算量：H×W×C×K×K（畳み込み層のチャネル数とフィルタ数分が削減）
  + Pointwise Convolution
    + 畳み込み層のカーネルは(1, 1, C)×Fでカーネルサイズを(1,1)に固定して処理
    + 畳み込み演算の計算量：H×W×C×F（カーネルサイズ(K,K)が削減）

  カーネルサイズとフィルタに関する演算を分割することで計算量を削減  
  H×W×C×(K×K×F - (K×K + F))  

<br>

【確認テスト】  
Depthwise Convolitionはチャネル毎に空間方向へ畳み込む。すなわち、チャネル毎にD_K×D_K×１のサイズのフィルターをそれぞれ用いて計算を行うため、その計算量は（？）となる。  
【解答】H×W×C×D_k×D_k 

<br>  

【確認テスト】  
次にDepthwise Convolutionの出力をPointwise Convolutionによってチャネル方向に畳み込む。すなわち、出力チャネル毎に１×１×Mサイズのフィルターをそれぞれ用いて計算を行うため、その計算量は（？）となる。  
【解答】H×W×C×M

+ DenseNet
  + Denseブロック  
    入力と畳み込み層との演算結果と入力データを別チャネルとして結合する(スキップコネクション）
    + バッチ正規化
    + 活性化関数(ReLU)
    + 畳み込み演算(3×3)  

    畳み込み演算する度に出力チャネル数分だけチャネル数が増加．  
  
  + Transition Layer: Denseブロックで増えたチャネル数を入力チャネル数に戻す処理が行われる
    + 畳み込み層(1,1)
    + AveragePooling層(2,2)  

  l層への入力をCl，初期入力チャネル数をk0，成長率(growth rate)をkとすると
  $$
  C_l = k_0 + k(l-1)
  $$
となる

  + ResNetとの違い
    + Denseブロック：前方各層の入力が後方層の入力となる⇔Ressidualブロック：前1層の入力のみ後方層の入力となる
    + Denseブロックにはgrowth rateのチャネル数だけ層が増加していく

+ Batch Normalization
  + レイヤに入力されるデータをミニバッチ単位で正規化(平均0，分散1)
  + 処理環境によってバッチサイズ(数枚～数十枚)が異るので評価しづらい
  + BatchNorm：サンプルデータ全体の同一チャネルが同一分布になるように正規化
    + ミニバッチのデータをチャネルごとにグループ化して正規化処理
    + バッチサイズが小さいと学習が収束しないことがある
  + LayerNorm：サンプルデータごとに同一分布になるように正規化
    + データ1枚ごとにチャネルを統合して正規化
    + バッチサイズによる影響はない
    + 入力データのスケール，重み行列のスケール・シフトにロバスト
  + InstanceNorm：各サンプルデータのチャネルごとに正規化
    + 画像のスタイル転送やテクスチャ合成で利用
+ Wavenet  
  音声を生成するモデル
  + 時系列データに対して畳み込みを適用：連続するデータに対してまとめてフィルタ処理をする
  + Dilated convolution：間隔を開けたデータでフィルタ処理をする  

<br>

【確認テスト】  

深層学習を用いて結合確率を学習する際に、効率的に学習が行えるアーキテクチャを提案したことがWaveNet の大きな貢献の1 つである。提案された新しいConvolution 型アーキテクチャは（？）と呼ばれ、結合確率を効率的に学習できるようになっている。  
【解答】Dilated causal convolution  

Depthwise separable convolution, Pointwise convolution ⇒ MoblieNets  

Deconvolution ⇒ 畳み込み処理の逆演算，オートエンコーダでも利用


【確認テスト】  
（？）を用いた際の大きな利点は、単純なConvolution layer と比べて（？？）ことである。  
【解答】パラメータ数に対する受容野が広い  
出力を長い時系列データを元に生成することができる


## Section5: Transformer
+ Transformer = seq2seq × attention => BERT
+ seq2seqとは系列データを入力して別の系列データを出力
  + Encoder-Decoderモデル
    + Encode：内部状態に変換
    + Decode：系列データに変換
  + 翻訳（日本語→英語）・音声認識（音声波形→テキスト）・チャットボット（テキスト→テキスト）
  + RNN：再帰的，時系列データをエンコード
    + 内部状態を保持している
  + 言語モデル：単語の並びに確率を与える（もっともらしさ：尤度）
    + 過去の時系列データで次の事象の出現確率分布を求める  

    ⇒学習がされていれば，先頭の単語を与えれば，文章が生成できる
  + seq2seq：EncoderのRNNとDecoderのRNNとの結合 
    + EncoderからDecoderに渡される内部状態が重要
    + Decoderの出力と正解ラベルで教師あり学習が行われる
  

+ BLEU：Bilingual Evaluation Understudy
  + 機械の翻訳が正解訳にどのくらい似ているのか
  $$
  BLEU = BP_{BLEU} \exp\left( \sum_{n=1}^{N} w_n \log p_n \right)\\
  w_n = \frac{1}{N}~~, ~~  p_n = \frac{\sum i \text{番目の学習データでの機械翻訳と正解訳で一致したn-gram数} }{\sum i \text{番目の学習データでの機械翻訳の全n-gram数}}\\
   BP_{BLEU} := \text{翻訳文が参照訳より短い場合のペナルティ}
  $$

+ seq2seq
【コード分析】  
参考文献を元に簡略化したseq2seqコードの構造を把握

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Study-AI/stage4/seq2seq/')

In [ ]:
import os
import random
import numpy as np

import random
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optimizers
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

from utils import Vocab

torch.manual_seed(1)
random_state = 42

```python
class Vocab(object):
    
    def __init__(self, word2id={}):
        """
        word2id: 単語(str)をインデックス(int)に変換する辞書
        id2word: インデックス(int)を単語(str)に変換する辞書
        """
        self.word2id = dict(word2id)
        self.id2word = {v: k for k, v in self.word2id.items()}    
        
    def build_vocab(self, sentences, min_count=1):
        # 各単語の出現回数の辞書を作成する
        word_counter = {}
        for sentence in sentences:
            for word in sentence:
                word_counter[word] = word_counter.get(word, 0) + 1

        # min_count回以上出現する単語のみ語彙に加える
        for word, count in sorted(word_counter.items(), key=lambda x: -x[1]):
            if count < min_count:
                break
            _id = len(self.word2id)
            self.word2id.setdefault(word, _id)
            self.id2word[_id] = word
```
+ Vocabクラス
  + インスタンス生成時に引数として渡した辞書オブジェクトを元に2種類の単語辞書を生成  
  （単語辞書に予約語を追加する）
  + build_vocabメソッド
    + 引数のsentencesは単語を要素とした2次元配列であることを想定
    + 1つ目のループでsentences全体での単語の出現回数を集計
    + 2つ目のループで引数で渡された値以上の出現回数の単語を  
    [word2id] キー：単語，　値：id番号  
    [id2word] キー：id番号，値：単語  
    で単語辞書を成長させる


In [ ]:
def load_data(file_path):
  data = []
  for line in open(file_path, encoding='utf-8'):
    words = line.strip().split()
    data.append(words)
  return data

In [ ]:
train_X = load_data('/content/drive/MyDrive/Study-AI/stage4/data/train.en')
train_Y = load_data('/content/drive/MyDrive/Study-AI/stage4/data/train.ja')

+ load_data関数
  + ファイルの内のテキストを単語単位の2次元配列に変換
  + 日本語テキストもスペース区切りの形式で渡されている

In [ ]:
train_X, valid_X, train_Y, valid_Y = train_test_split(train_X, train_Y, test_size=0.2, random_state=random_state)

In [ ]:
PAD_TOKEN, BOS_TOKEN, EOS_TOKEN, UNK_TOKEN = '<PAD>', '<S>', '</S>', '<UNK>'
PAD, BOS, EOS, UNK = 0, 1, 2, 3

MIN_COUNT = 2  
word2id = {PAD_TOKEN: PAD, BOS_TOKEN: BOS, EOS_TOKEN: EOS, UNK_TOKEN: UNK}

vocab_X = Vocab(word2id=word2id)
vocab_Y = Vocab(word2id=word2id)
vocab_X.build_vocab(train_X, min_count=MIN_COUNT)
vocab_Y.build_vocab(train_Y, min_count=MIN_COUNT)

+ 英語と日本語の単語辞書を生成
+ 予約語は4つ
+ 文書ファイル内に2回以上出現する単語を辞書に登録

In [ ]:
def sentence_to_ids(vocab, sentence):
  ids = [vocab.word2id.get(word, UNK) for word in sentence]
  ids += [EOS]
  return ids

In [ ]:
train_X = [sentence_to_ids(vocab_X, sentence) for sentence in train_X]
train_Y = [sentence_to_ids(vocab_Y, sentence) for sentence in train_Y]
valid_X = [sentence_to_ids(vocab_X, sentence) for sentence in valid_X]
valid_Y = [sentence_to_ids(vocab_Y, sentence) for sentence in valid_Y]

+ sentence_to_ids関数
  + 単語辞書vocabを基にテキストデータのsentenceをid番号化した配列に変換
  + 文末に[EOS]を追加
+ 英語と日本語の文書について学習用と検証用それぞれに対しid番号化する

In [ ]:
def pad_seq(seq, max_length):
  res = seq + [PAD for i in range(max_length - len(seq))]
  return res    

+ pad_seq関数
  + 各単語列の要素数を統一するためpadding

In [ ]:
from sklearn.utils import shuffle

class DataLoader(object):

  def __init__(self, X, Y, batch_size, shuffle=False):
    self.data = list(zip(X, Y))
    self.batch_size = batch_size
    self.shuffle = shuffle
    self.start_index = 0
    
    self.reset()
  
  def reset(self):
    if self.shuffle:
      self.data = shuffle(self.data, random_state=random_state)
    self.start_index = 0
  
  def __iter__(self):
    return self

  def __next__(self):
    if self.start_index >= len(self.data):
      self.reset()
      raise StopIteration()

    seqs_X, seqs_Y = zip(*self.data[self.start_index:self.start_index+self.batch_size])

    seq_pairs = sorted(zip(seqs_X, seqs_Y), key=lambda p: len(p[0]), reverse=True)
    seqs_X, seqs_Y = zip(*seq_pairs)
    lengths_X = [len(s) for s in seqs_X]
    lengths_Y = [len(s) for s in seqs_Y]
    max_length_X = max(lengths_X)
    max_length_Y = max(lengths_Y)
    padded_X = [pad_seq(s, max_length_X) for s in seqs_X]
    padded_Y = [pad_seq(s, max_length_Y) for s in seqs_Y]

    batch_X = torch.tensor(padded_X, dtype=torch.long).transpose(0, 1)
    batch_Y = torch.tensor(padded_Y, dtype=torch.long).transpose(0, 1)

    self.start_index += self.batch_size

    return batch_X, batch_Y, lengths_X

+ DataLoaderクラス
  + バッチサイズの指定が可能
  + データのシャッフルが可能
  + 学習モデルで利用可能なデータ形式を出力
    + \__next\__メソッド内
      + 英語文と日本語訳との対はzip関数で保持したままの変数dataを基本に処理
      + 単語配列のデータ長を揃えるためにpadding処理をするが，処理の効率化（padがより少なくなるように）のためにバッチ内でのデータ長のばらつきを小さくするためデータ長でソート
      + バッチ単位でpadding処理しデータ長を揃える
      + 学習モデルで利用可能な形式に変換

In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = nn.Embedding(input_size, hidden_size, padding_idx=PAD)
    self.gru = nn.GRU(hidden_size, hidden_size)

  def forward(self, seqs, input_lengths, hidden=None):
    emb = self.embedding(seqs)
    packed = pack_padded_sequence(emb, input_lengths)
    output, hidden = self.gru(packed, hidden)
    output, _ = pad_packed_sequence(output)
    return output, hidden

+ Encoderクラス
  + インスタンス生成時のパラメータに対応したEmbedding行列とGRUユニットを生成
  + forwardメソッド（順伝播）
    + 入力単語列を分散表現に変換
    + pack_padded_sequence関数で入力データをpadding前の単語列データに変換
    + GRUユニットで演算で，出力データと隠れ層データを生成
    + pad_packed_sequence関数で出力データをpadding後の単語列データに変換
    + padding後の出力データと隠れ層データを戻す


In [ ]:
class Decoder(nn.Module):
  def __init__(self, hidden_size, output_size):
    super(Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size

    self.embedding = nn.Embedding(output_size, hidden_size, padding_idx=PAD)
    self.gru = nn.GRU(hidden_size, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)

  def forward(self, seqs, hidden):
    emb = self.embedding(seqs)
    output, hidden = self.gru(emb, hidden)
    output = self.out(output)
    return output, hidden

+ Decoderクラス
  + インスタンス生成時のパラメータに対応したEmbedding行列，GRUユニット，線形ユニットを生成
  + forwardメソッド（順伝播）
    + 入力データを分散表現に変換
    + GRUユニットで出力データと隠れ層データを生成
    + 出力データを線形ユニットで演算
    + 出力データと隠れ層データを戻す

In [ ]:
class EncoderDecoder(nn.Module):
  def __init__(self, input_size, output_size, hidden_size):
    super(EncoderDecoder, self).__init__()
    self.encoder = Encoder(input_size, hidden_size)
    self.decoder = Decoder(hidden_size, output_size)

  def forward(self, batch_X, lengths_X, max_length, batch_Y=None, use_teacher_forcing=False):
    _, encoder_hidden = self.encoder(batch_X, lengths_X)
    _batch_size = batch_X.size(1)

    decoder_input = torch.tensor([BOS] * _batch_size, dtype=torch.long)
    decoder_input = decoder_input.unsqueeze(0)
    decoder_hidden = encoder_hidden

    decoder_outputs = torch.zeros(max_length, _batch_size, self.decoder.output_size)

    for t in range(max_length):
      decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
      decoder_outputs[t] = decoder_output
      if use_teacher_forcing and batch_Y is not None:
        decoder_input = batch_Y[t].unsqueeze(0)
      else:
        decoder_input = decoder_output.max(-1)[1]
        
    return decoder_outputs

+ EncoderDecoderクラス
  + EncoderとDecoderのインスタンスを1つずつ生成
  + forwardメソッド
    + エンコーダインスタンスで入力データをエンコード処理し，デコーダインスタンスに渡す隠れ層データを生成
    + デコーダインスタンスで利用する初期データを生成
    + ループ処理内でデコード処理
    + use_teacher_forcingを有効にしていた場合はループの入力データに正解データを使う
    + デコード処理結果を戻す

In [ ]:
mce = nn.CrossEntropyLoss(reduction='sum', ignore_index=PAD)
def masked_cross_entropy(logits, target):
  logits_flat = logits.view(-1, logits.size(-1))
  target_flat = target.view(-1)
  return mce(logits_flat, target_flat)

+ masked_cross_entropy関数
  + 予測データと正解データを引数としてクロスエントロピー誤差を計算
  + データのサイズ変更に際し，メモリー上で要素順に並べる処理(contiguous)後のデータを受け取ることを想定
  + paddingの要素は無視して演算

In [ ]:
vocab_size_X = len(vocab_X.id2word)
vocab_size_Y = len(vocab_Y.id2word)

In [ ]:
num_epochs = 10
batch_size = 64
lr = 1e-3
teacher_forcing_rate = 0.2
ckpt_path = 'model.pth'

model_args = {'input_size': vocab_size_X, 'output_size': vocab_size_Y, 'hidden_size': 256}

In [ ]:
def compute_loss(batch_X, batch_Y, lengths_X, model, optimizer=None, is_train=True):
  model.train(is_train)
  
  use_teacher_forcing = is_train and (random.random() < teacher_forcing_rate)
  max_length = batch_Y.size(0)

  pred_Y = model(batch_X, lengths_X, max_length, batch_Y, use_teacher_forcing)

  loss = masked_cross_entropy(pred_Y.contiguous(), batch_Y.contiguous())
  
  if is_train:
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  batch_Y = batch_Y.transpose(0, 1).contiguous().data.cpu().tolist()
  pred = pred_Y.max(dim=-1)[1].data.cpu().numpy().T.tolist()

  return loss.item(), batch_Y, pred

+ compute_loss関数
  + クロスエントロピー誤差の演算処理
  + 学習モードでパラメータ更新を実行
  + 戻り値のうち，今回利用するのはloss.item()のみ
  + 戻り値のbatch_YとpredはBLEU計算に利用

In [ ]:
train_dataloader = DataLoader(train_X, train_Y, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_X, valid_Y, batch_size=batch_size, shuffle=False)

model = EncoderDecoder(**model_args)
optimizer = optimizers.Adam(model.parameters(), lr=lr)

+ 学習用データの生成
  + 学習用データと評価用データをバッチサイズに合わせて生成
+ 学習モデル(EncoderDecoder)の生成
+ 最適化手法をAdamに設定

In [ ]:
for epoch in range(1, num_epochs+1):
  train_loss = 0.
  #train_refs = []
  #train_hyps = []
  valid_loss = 0.
  #valid_refs = []
  #valid_hyps = []

  for batch in train_dataloader:
    batch_X, batch_Y, lengths_X = batch
    loss, gold, pred = compute_loss(
      batch_X, batch_Y, lengths_X, model, optimizer, 
      is_train=True
      )
    train_loss += loss
    #train_refs += gold
    #train_hyps += pred

  for batch in valid_dataloader:
    batch_X, batch_Y, lengths_X = batch
    loss, gold, pred = compute_loss(
      batch_X, batch_Y, lengths_X, model, 
      is_train=False
      )
    valid_loss += loss
    #valid_refs += gold
    #valid_hyps += pred

  train_loss = np.sum(train_loss) / len(train_dataloader.data)
  valid_loss = np.sum(valid_loss) / len(valid_dataloader.data)


  print('Epoch {}: train_loss: {:5.2f} valid_loss: {:5.2f}'.format(
      epoch, train_loss, valid_loss))
  print('-'*40)

Epoch 1: train_loss: 52.09 valid_loss: 48.95
----------------------------------------
Epoch 2: train_loss: 44.58 valid_loss: 44.80
----------------------------------------
Epoch 3: train_loss: 40.05 valid_loss: 42.64
----------------------------------------
Epoch 4: train_loss: 37.57 valid_loss: 41.01
----------------------------------------
Epoch 5: train_loss: 35.00 valid_loss: 40.30
----------------------------------------
Epoch 6: train_loss: 33.25 valid_loss: 40.07
----------------------------------------
Epoch 7: train_loss: 31.83 valid_loss: 40.13
----------------------------------------
Epoch 8: train_loss: 30.61 valid_loss: 39.98
----------------------------------------
Epoch 9: train_loss: 29.09 valid_loss: 40.14
----------------------------------------
Epoch 10: train_loss: 28.15 valid_loss: 40.42
----------------------------------------


+ 学習
  + ミニバッチ用学習データでパラメータ更新と誤差の計算を実施
  + ミニバッチ用評価データで学習後のモデルの誤差を計算

【参考文献】
1. 斎藤康毅著『セロから作るDeepLearning2 自然言語処理編』オライリー・ジャパン 2018.07
2. 巣籠悠輔著『詳解ディープラーニング 第2版』マイナビ出版 2019.11

+ Attention
  + seq2seqはデータを固定長ベクトルに変換するので，長い文章に弱い
  $$
  \boldsymbol{c} = f(\boldsymbol{h}_s (t-1), \boldsymbol{x}(t))\\
  \boldsymbol{h}_t (t) = f(\boldsymbol{h}_t (t-1), \boldsymbol{y}(t-1), \boldsymbol{c})
  $$
  + 時刻ごとに生成される隠れ状態に重みをつける⇒Attention⇒Transformer
  $$
  \boldsymbol{c}(t) = \sum_{\tau=1}^{T} a(\tau, t)\boldsymbol{h}_s(\tau)~~, ~~
  \sum_{\tau=1}^{T} a(\tau, t) = 1\\
  \boldsymbol{h}_t (t) = f(\boldsymbol{h}_t (t-1), \boldsymbol{y}(t-1), \color {red}{\boldsymbol{c}(t)})
  $$
  ※時刻tで，各時刻で生成されるhsの重要度(重み)が異なることを表現できる  
  ※重要度(重み)=注意Attentionでありこの値をどのように計算するか
$$
a(\tau, t) = softmax(g(\boldsymbol{h}_s (\tau),\boldsymbol{h}_t (t-1)))
$$
  + Attentionはquery・key・valueの三つ組の構成
    + keyに対応するqueryからvalueを取り出す  
Scaled Dot-Product Attention  
$$
Attention(Q, K, V) = softmax \left(\frac{QK^T}{\sqrt{d_k}} \right ) V
$$

  + 長い文章でも性能が出せるようになる
    + ソース・ターゲット：queryはtargetデータ，ほかはsourceデータ
    + Self-Attention：queryもsourceデータ  

$$ 
g:=\left \{ 
\begin{array}{l}
\boldsymbol{h}_t^T \boldsymbol{h}_s \\
\boldsymbol{h}_s^T \boldsymbol{h}_s
\end{array}\right .
$$
+ Transformer(Self-Attention)
  + Encoder Decoderの構造だが，
  + RNNを利用せずAttentionモジュールで構成
    + 時系列ではないので単語の語順を保持できない⇒Positional Encoding
    + デコーダで未来の単語を利用しない⇒Masked Multi-Head Attention
  + Feed Forwardモジュール
    + 2つの全結合層と間に活性化関数ReLUを使ったもモジュール
$$
u = x W_1 + b_1 \\
z = relu(u)\\
y = z W_2 + b_2
$$
  + Positional Encodingモジュール
    + 入力データが時系列ではにので順序情報を付加する
    + 入力データの次元数と特徴量次元数からなるマトリクスで対応
      + 特徴量のインデックが偶数のときはsin値，奇数のときはcos値  
$$
PE_{(pos, 2i)} = \sin \left (\frac{pos}{10000^{\frac{2i}{d}}} \right)~~,~~
PE_{(pos, 2i+1)} = \cos \left (\frac{pos}{10000^{\frac{2i}{d}}} \right)
$$
  
  + Multi-Head Attention
    + Scaled Dot-Product Attentionを複数並列処理
    + アンサンブル学習のような効果が期待できる
  + Masked Multi-Head Attention
    + デコーダ内で時刻t以降の単語情報を使用しないようにMaskをかけたもの
    + 動作自体はMulit-Head Attentionと同様
  + （Add&Normモジュール）
    + residual connectionと正規化処理を実行 


【コード分析】

In [ ]:
def position_encoding_init(n_position, d_pos_vec):
    """
    Positional Encodingのための行列の初期化を行う
    :param n_position: int, 系列長
    :param d_pos_vec: int, 隠れ層の次元数
    :return torch.tensor, size=(n_position, d_pos_vec)
    """
    # PADがある単語の位置はpos=0にしておき、position_encも0にする
    position_enc = np.array([
        [pos / np.power(10000, 2 * (j // 2) / d_pos_vec) for j in range(d_pos_vec)]
        if pos != 0 else np.zeros(d_pos_vec) for pos in range(n_position)])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return torch.tensor(position_enc, dtype=torch.float)

+ position_encoding_init関数
  + 入力データと特徴量のマトリクスを生成
    + 要素は三角関数の引数になる値を生成
  + マトリクスの偶数列はsin，奇数列はcosで演算し要素を上書き

In [ ]:
class ScaledDotProductAttention(nn.Module):
    
    def __init__(self, d_model, attn_dropout=0.1):
        """
        :param d_model: int, 隠れ層の次元数
        :param attn_dropout: float, ドロップアウト率
        """
        super(ScaledDotProductAttention, self).__init__()
        self.temper = np.power(d_model, 0.5)  # スケーリング因子
        self.dropout = nn.Dropout(attn_dropout)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v, attn_mask):
        """
        :param q: torch.tensor, queryベクトル, 
            size=(n_head*batch_size, len_q, d_model/n_head)
        :param k: torch.tensor, key, 
            size=(n_head*batch_size, len_k, d_model/n_head)
        :param v: torch.tensor, valueベクトル, 
            size=(n_head*batch_size, len_v, d_model/n_head)
        :param attn_mask: torch.tensor, Attentionに適用するマスク, 
            size=(n_head*batch_size, len_q, len_k)
        :return output: 出力ベクトル, 
            size=(n_head*batch_size, len_q, d_model/n_head)
        :return attn: Attention
            size=(n_head*batch_size, len_q, len_k)
        """
        # QとKの内積でAttentionの重みを求め、スケーリングする
        attn = torch.bmm(q, k.transpose(1, 2)) / self.temper  # (n_head*batch_size, len_q, len_k)
        # Attentionをかけたくない部分がある場合は、その部分を負の無限大に飛ばしてSoftmaxの値が0になるようにする
        attn.data.masked_fill_(attn_mask, -float('inf'))
        
        attn = self.softmax(attn)
        attn = self.dropout(attn)
        output = torch.bmm(attn, v)

        return output, attn

+ ScaledDotProductAttention クラス
  + temper: 隠れ層次元数の平方根
  + torch.bmm: 3次元配列同士の演算の場合，(b,n,m) @ (b,m,p)⇒(b,n,p)  
  bのインデックス番号ごとに(m,m)と(m,p)の内積を計算
  + torch.tensor.masked_fill_: 単語の入力がない要素は-∞で埋める
  + softmaxモジュールを通す
  + dropoutモジュールを通す
  + Attention配列とvalue配列とのbmm演算を出力とする
  + 出力配列とAttention配列を返す

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):
        """
        :param n_head: int, ヘッド数
        :param d_model: int, 隠れ層の次元数
        :param d_k: int, keyベクトルの次元数
        :param d_v: int, valueベクトルの次元数
        :param dropout: float, ドロップアウト率
        """
        super(MultiHeadAttention, self).__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        # 各ヘッドごとに異なる重みで線形変換を行うための重み
        # nn.Parameterを使うことで、Moduleのパラメータとして登録できる. TFでは更新が必要な変数はtf.Variableでラップするのでわかりやすい
        self.w_qs = nn.Parameter(torch.empty([n_head, d_model, d_k], dtype=torch.float))
        self.w_ks = nn.Parameter(torch.empty([n_head, d_model, d_k], dtype=torch.float))
        self.w_vs = nn.Parameter(torch.empty([n_head, d_model, d_v], dtype=torch.float))
        # nn.init.xavier_normal_で重みの値を初期化
        nn.init.xavier_normal_(self.w_qs)
        nn.init.xavier_normal_(self.w_ks)
        nn.init.xavier_normal_(self.w_vs)

        self.attention = ScaledDotProductAttention(d_model)
        self.layer_norm = nn.LayerNorm(d_model) # 各層においてバイアスを除く活性化関数への入力を平均０、分散１に正則化
        self.proj = nn.Linear(n_head*d_v, d_model)  # 複数ヘッド分のAttentionの結果を元のサイズに写像するための線形層
        # nn.init.xavier_normal_で重みの値を初期化
        nn.init.xavier_normal_(self.proj.weight)
        
        self.dropout = nn.Dropout(dropout)


    def forward(self, q, k, v, attn_mask=None):
        """
        :param q: torch.tensor, queryベクトル, 
            size=(batch_size, len_q, d_model)
        :param k: torch.tensor, key, 
            size=(batch_size, len_k, d_model)
        :param v: torch.tensor, valueベクトル, 
            size=(batch_size, len_v, d_model)
        :param attn_mask: torch.tensor, Attentionに適用するマスク, 
            size=(batch_size, len_q, len_k)
        :return outputs: 出力ベクトル, 
            size=(batch_size, len_q, d_model)
        :return attns: Attention
            size=(n_head*batch_size, len_q, len_k)
            
        """
        d_k, d_v = self.d_k, self.d_v
        n_head = self.n_head

        # residual connectionのための入力 出力に入力をそのまま加算する
        residual = q

        batch_size, len_q, d_model = q.size()
        batch_size, len_k, d_model = k.size()
        batch_size, len_v, d_model = v.size()

        # 複数ヘッド化
        # torch.repeat または .repeatで指定したdimに沿って同じテンソルを作成
        q_s = q.repeat(n_head, 1, 1) # (n_head*batch_size, len_q, d_model)
        k_s = k.repeat(n_head, 1, 1) # (n_head*batch_size, len_k, d_model)
        v_s = v.repeat(n_head, 1, 1) # (n_head*batch_size, len_v, d_model)
        # ヘッドごとに並列計算させるために、n_headをdim=0に、batch_sizeをdim=1に寄せる
        q_s = q_s.view(n_head, -1, d_model) # (n_head, batch_size*len_q, d_model)
        k_s = k_s.view(n_head, -1, d_model) # (n_head, batch_size*len_k, d_model)
        v_s = v_s.view(n_head, -1, d_model) # (n_head, batch_size*len_v, d_model)

        # 各ヘッドで線形変換を並列計算(p16左側`Linear`)
        q_s = torch.bmm(q_s, self.w_qs)  # (n_head, batch_size*len_q, d_k)
        k_s = torch.bmm(k_s, self.w_ks)  # (n_head, batch_size*len_k, d_k)
        v_s = torch.bmm(v_s, self.w_vs)  # (n_head, batch_size*len_v, d_v)
        # Attentionは各バッチ各ヘッドごとに計算させるためにbatch_sizeをdim=0に寄せる
        q_s = q_s.view(-1, len_q, d_k)   # (n_head*batch_size, len_q, d_k)
        k_s = k_s.view(-1, len_k, d_k)   # (n_head*batch_size, len_k, d_k)
        v_s = v_s.view(-1, len_v, d_v)   # (n_head*batch_size, len_v, d_v)

        # Attentionを計算(p16.左側`Scaled Dot-Product Attention * h`)
        outputs, attns = self.attention(q_s, k_s, v_s, attn_mask=attn_mask.repeat(n_head, 1, 1))

        # 各ヘッドの結果を連結(p16左側`Concat`)
        # torch.splitでbatch_sizeごとのn_head個のテンソルに分割
        outputs = torch.split(outputs, batch_size, dim=0)  # (batch_size, len_q, d_model) * n_head
        # dim=-1で連結
        outputs = torch.cat(outputs, dim=-1)  # (batch_size, len_q, d_model*n_head)

        # residual connectionのために元の大きさに写像(p16左側`Linear`)
        outputs = self.proj(outputs)  # (batch_size, len_q, d_model)
        outputs = self.dropout(outputs)
        outputs = self.layer_norm(outputs + residual)

        return outputs, attns

+  MultiHeadAttentionクラス
  + Add&Normモジュールの機能も含む
  + attn_mask引数の有無で処理が分かれる(Masked or Non Masked)
  + ScaledDotProductAttentionクラスのインスタンスを生成し，
  + 入力するquery，key, valueを複製
  + 複製したquery, key, valueをまとめてインスタンスに渡す
  + 出力配列をバッチサイズで分割
  + 出力配列を結合し直す
  + 出力配列を入力配列のサイズに戻す
  + 入力配列と出力配列の和に対し正規化処理を実施
  + 出力配列とAttention配列を返す

In [ ]:
class PositionwiseFeedForward(nn.Module):
    """
    :param d_hid: int, 隠れ層1層目の次元数
    :param d_inner_hid: int, 隠れ層2層目の次元数
    :param dropout: float, ドロップアウト率
    """
    def __init__(self, d_hid, d_inner_hid, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        # window size 1のconv層を定義することでPosition wiseな全結合層を実現する.
        self.w_1 = nn.Conv1d(d_hid, d_inner_hid, 1)
        self.w_2 = nn.Conv1d(d_inner_hid, d_hid, 1)
        self.layer_norm = nn.LayerNorm(d_hid)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        :param x: torch.tensor,
            size=(batch_size, max_length, d_hid)
        :return: torch.tensor,
            size=(batch_size, max_length, d_hid) 
        """
        residual = x
        output = self.relu(self.w_1(x.transpose(1, 2)))
        output = self.w_2(output).transpose(2, 1)
        output = self.dropout(output)
        return self.layer_norm(output + residual)

+  PositionwiseFeedForwardクラス
  + Add&Normモジュールの機能も含む
  + 入力配列を全結合層に通し，活性化関数(ReLU)を通す
  + 出力配列を全結合層に通す
  + 入力配列と出力配列の和を正規化処理したものを返す

In [ ]:
class EncoderLayer(nn.Module):
    """Encoderのブロックのクラス"""
    def __init__(self, d_model, d_inner_hid, n_head, d_k, d_v, dropout=0.1):
        """
        :param d_model: int, 隠れ層の次元数
        :param d_inner_hid: int, Position Wise Feed Forward Networkの隠れ層2層目の次元数
        :param n_head: int,　ヘッド数
        :param d_k: int, keyベクトルの次元数
        :param d_v: int, valueベクトルの次元数
        :param dropout: float, ドロップアウト率
        """
        super(EncoderLayer, self).__init__()
        # Encoder内のSelf-Attention
        self.slf_attn = MultiHeadAttention(
            n_head, d_model, d_k, d_v, dropout=dropout)
        # Postionwise FFN
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner_hid, dropout=dropout)

    def forward(self, enc_input, slf_attn_mask=None):
        """
        :param enc_input: tensor, Encoderの入力, 
            size=(batch_size, max_length, d_model)
        :param slf_attn_mask: tensor, Self Attentionの行列にかけるマスク, 
            size=(batch_size, len_q, len_k)
        :return enc_output: tensor, Encoderの出力, 
            size=(batch_size, max_length, d_model)
        :return enc_slf_attn: tensor, EncoderのSelf Attentionの行列, 
            size=(n_head*batch_size, len_q, len_k)
        """
        # Self-Attentionのquery, key, valueにはすべてEncoderの入力（enc_input）が入る
        enc_output, enc_slf_attn = self.slf_attn(
            enc_input, enc_input, enc_input, attn_mask=slf_attn_mask)
        enc_output = self.pos_ffn(enc_output)
        return enc_output, enc_slf_attn

In [ ]:
class Encoder(nn.Module):
    """EncoderLayerブロックからなるEncoderのクラス"""
    def __init__(
            self, n_src_vocab, max_length, n_layers=6, n_head=8, d_k=64, d_v=64,
            d_word_vec=512, d_model=512, d_inner_hid=1024, dropout=0.1):
        """
        :param n_src_vocab: int, 入力言語の語彙数
        :param max_length: int, 最大系列長
        :param n_layers: int, レイヤー数
        :param n_head: int,　ヘッド数
        :param d_k: int, keyベクトルの次元数
        :param d_v: int, valueベクトルの次元数
        :param d_word_vec: int, 単語の埋め込みの次元数
        :param d_model: int, 隠れ層の次元数
        :param d_inner_hid: int, Position Wise Feed Forward Networkの隠れ層2層目の次元数
        :param dropout: float, ドロップアウト率        
        """
        super(Encoder, self).__init__()

        n_position = max_length + 1
        self.max_length = max_length
        self.d_model = d_model

        # Positional Encodingを用いたEmbedding
        self.position_enc = nn.Embedding(n_position, d_word_vec, padding_idx=PAD)
        self.position_enc.weight.data = position_encoding_init(n_position, d_word_vec)

        # 一般的なEmbedding
        self.src_word_emb = nn.Embedding(n_src_vocab, d_word_vec, padding_idx=PAD)

        # EncoderLayerをn_layers個積み重ねる
        self.layer_stack = nn.ModuleList([
            EncoderLayer(d_model, d_inner_hid, n_head, d_k, d_v, dropout=dropout)
            for _ in range(n_layers)])

    def forward(self, src_seq, src_pos):
        """
        :param src_seq: tensor, 入力系列, 
            size=(batch_size, max_length)
        :param src_pos: tensor, 入力系列の各単語の位置情報,
            size=(batch_size, max_length)
        :return enc_output: tensor, Encoderの最終出力, 
            size=(batch_size, max_length, d_model)
        :return enc_slf_attns: list, EncoderのSelf Attentionの行列のリスト
        """
        # 一般的な単語のEmbeddingを行う
        enc_input = self.src_word_emb(src_seq)
        # Positional EncodingのEmbeddingを加算する
        enc_input += self.position_enc(src_pos)

        enc_slf_attns = []
        enc_output = enc_input
        # key(=enc_input)のPADに対応する部分のみ1のマスクを作成
        enc_slf_attn_mask = get_attn_padding_mask(src_seq, src_seq)

        # n_layers個のEncoderLayerに入力を通す
        for enc_layer in self.layer_stack:
            enc_output, enc_slf_attn = enc_layer(
                enc_output, slf_attn_mask=enc_slf_attn_mask)
            enc_slf_attns += [enc_slf_attn]

        return enc_output, enc_slf_attns

+ EncoderLayerクラス
  + MultiHeadAttentionとPositionwiseFeedForwardを結合したクラス
  + Self-Attentionで処理
+ Encoderクラス
  + エンコーダ部分をまとめたクラス
  + 入力系列に対しEmbedding
  + Positional Encodingを加算
  + PAD部分にマスク処理
  + EncoderLayerの処理を複数回処理

In [ ]:
class DecoderLayer(nn.Module):
    """Decoderのブロックのクラス"""
    def __init__(self, d_model, d_inner_hid, n_head, d_k, d_v, dropout=0.1):
        """
        :param d_model: int, 隠れ層の次元数
        :param d_inner_hid: int, Position Wise Feed Forward Networkの隠れ層2層目の次元数
        :param n_head: int,　ヘッド数
        :param d_k: int, keyベクトルの次元数
        :param d_v: int, valueベクトルの次元数
        :param dropout: float, ドロップアウト率
        """
        super(DecoderLayer, self).__init__()
        # Decoder内のSelf-Attention
        self.slf_attn = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
        # Encoder-Decoder間のSource-Target Attention
        self.enc_attn = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
        # Positionwise FFN
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner_hid, dropout=dropout)

    def forward(self, dec_input, enc_output, slf_attn_mask=None, dec_enc_attn_mask=None):
        """
        :param dec_input: tensor, Decoderの入力, 
            size=(batch_size, max_length, d_model)
        :param enc_output: tensor, Encoderの出力, 
            size=(batch_size, max_length, d_model)
        :param slf_attn_mask: tensor, Self Attentionの行列にかけるマスク, 
            size=(batch_size, len_q, len_k)
        :param dec_enc_attn_mask: tensor, Soutce-Target Attentionの行列にかけるマスク, 
            size=(batch_size, len_q, len_k)
        :return dec_output: tensor, Decoderの出力, 
            size=(batch_size, max_length, d_model)
        :return dec_slf_attn: tensor, DecoderのSelf Attentionの行列, 
            size=(n_head*batch_size, len_q, len_k)
        :return dec_enc_attn: tensor, DecoderのSoutce-Target Attentionの行列, 
            size=(n_head*batch_size, len_q, len_k)
        """
        # Self-Attentionのquery, key, valueにはすべてDecoderの入力（dec_input）が入る
        dec_output, dec_slf_attn = self.slf_attn(
            dec_input, dec_input, dec_input, attn_mask=slf_attn_mask)
        # Source-Target-AttentionのqueryにはDecoderの出力(dec_output), key, valueにはEncoderの出力（enc_output）が入る
        dec_output, dec_enc_attn = self.enc_attn(
            dec_output, enc_output, enc_output, attn_mask=dec_enc_attn_mask)
        dec_output = self.pos_ffn(dec_output)

        return dec_output, dec_slf_attn, dec_enc_attn

+ DecoderLayerクラス
  + 2種類のMultiHeadAttentionとPositionwiseFeedForwardを結合したクラス
    + エンコーダからの出力はSource-Target Attention
    + デコーダへの入力はSelf-Attention
    + 出力系列，Self-Attention, Source-Target Attentionそれぞれに対応するデータを出力

In [ ]:
class Decoder(nn.Module):
    """DecoderLayerブロックからなるDecoderのクラス"""
    def __init__(
            self, n_tgt_vocab, max_length, n_layers=6, n_head=8, d_k=64, d_v=64,
            d_word_vec=512, d_model=512, d_inner_hid=1024, dropout=0.1):
        """
        :param n_tgt_vocab: int, 出力言語の語彙数
        :param max_length: int, 最大系列長
        :param n_layers: int, レイヤー数
        :param n_head: int,　ヘッド数
        :param d_k: int, keyベクトルの次元数
        :param d_v: int, valueベクトルの次元数
        :param d_word_vec: int, 単語の埋め込みの次元数
        :param d_model: int, 隠れ層の次元数
        :param d_inner_hid: int, Position Wise Feed Forward Networkの隠れ層2層目の次元数
        :param dropout: float, ドロップアウト率        
        """
        super(Decoder, self).__init__()
        n_position = max_length + 1
        self.max_length = max_length
        self.d_model = d_model

        # Positional Encodingを用いたEmbedding
        self.position_enc = nn.Embedding(
            n_position, d_word_vec, padding_idx=PAD)
        self.position_enc.weight.data = position_encoding_init(n_position, d_word_vec)

        # 一般的なEmbedding
        self.tgt_word_emb = nn.Embedding(
            n_tgt_vocab, d_word_vec, padding_idx=PAD)
        self.dropout = nn.Dropout(dropout)

        # DecoderLayerをn_layers個積み重ねる
        self.layer_stack = nn.ModuleList([
            DecoderLayer(d_model, d_inner_hid, n_head, d_k, d_v, dropout=dropout)
            for _ in range(n_layers)])

    def forward(self, tgt_seq, tgt_pos, src_seq, enc_output):
        """
        :param tgt_seq: tensor, 出力系列, 
            size=(batch_size, max_length)
        :param tgt_pos: tensor, 出力系列の各単語の位置情報,
            size=(batch_size, max_length)
        :param src_seq: tensor, 入力系列, 
            size=(batch_size, n_src_vocab)
        :param enc_output: tensor, Encoderの出力, 
            size=(batch_size, max_length, d_model)
        :return dec_output: tensor, Decoderの最終出力, 
            size=(batch_size, max_length, d_model)
        :return dec_slf_attns: list, DecoderのSelf Attentionの行列のリスト 
        :return dec_slf_attns: list, DecoderのSelf Attentionの行列のリスト
        """
        # 一般的な単語のEmbeddingを行う
        dec_input = self.tgt_word_emb(tgt_seq)
        # Positional EncodingのEmbeddingを加算する
        dec_input += self.position_enc(tgt_pos)

        # Self-Attention用のマスクを作成
        # key(=dec_input)のPADに対応する部分が1のマスクと、queryから見たkeyの未来の情報に対応する部分が1のマスクのORをとる
        dec_slf_attn_pad_mask = get_attn_padding_mask(tgt_seq, tgt_seq)  # (N, max_length, max_length)
        dec_slf_attn_sub_mask = get_attn_subsequent_mask(tgt_seq)  # (N, max_length, max_length)
        dec_slf_attn_mask = torch.gt(dec_slf_attn_pad_mask + dec_slf_attn_sub_mask, 0)  # ORをとる

        # key(=dec_input)のPADに対応する部分のみ1のマスクを作成
        dec_enc_attn_pad_mask = get_attn_padding_mask(tgt_seq, src_seq)  # (N, max_length, max_length)

        dec_slf_attns, dec_enc_attns = [], []

        dec_output = dec_input
        # n_layers個のDecoderLayerに入力を通す
        for dec_layer in self.layer_stack:
            dec_output, dec_slf_attn, dec_enc_attn = dec_layer(
                dec_output, enc_output,
                slf_attn_mask=dec_slf_attn_mask,
                dec_enc_attn_mask=dec_enc_attn_pad_mask)

            dec_slf_attns += [dec_slf_attn]
            dec_enc_attns += [dec_enc_attn]

        return dec_output, dec_slf_attns, dec_enc_attns

+ Decoderクラス
  + デコーダ部分をまとめたクラス
  + 入力系列に対しEmbedding
  + Positional Encodingを加算
  + PAD部分にマスク処理
  + 未来の単語をマスク処理
  + マスク処理した2つのデータをOR演算
  + エンコーダからの入力系列にマスク処理
  + DecoderLayerの処理を複数回処理

In [ ]:
class Transformer(nn.Module):
    """Transformerのモデル全体のクラス"""
    def __init__(
            self, n_src_vocab, n_tgt_vocab, max_length, n_layers=6, n_head=8,
            d_word_vec=512, d_model=512, d_inner_hid=1024, d_k=64, d_v=64,
            dropout=0.1, proj_share_weight=True):
        """
        :param n_src_vocab: int, 入力言語の語彙数
        :param n_tgt_vocab: int, 出力言語の語彙数
        :param max_length: int, 最大系列長
        :param n_layers: int, レイヤー数
        :param n_head: int,　ヘッド数
        :param d_k: int, keyベクトルの次元数
        :param d_v: int, valueベクトルの次元数
        :param d_word_vec: int, 単語の埋め込みの次元数
        :param d_model: int, 隠れ層の次元数
        :param d_inner_hid: int, Position Wise Feed Forward Networkの隠れ層2層目の次元数
        :param dropout: float, ドロップアウト率        
        :param proj_share_weight: bool, 出力言語の単語のEmbeddingと出力の写像で重みを共有する        
        """
        super(Transformer, self).__init__()
        self.encoder = Encoder(
            n_src_vocab, max_length, n_layers=n_layers, n_head=n_head,
            d_word_vec=d_word_vec, d_model=d_model,
            d_inner_hid=d_inner_hid, dropout=dropout)
        self.decoder = Decoder(
            n_tgt_vocab, max_length, n_layers=n_layers, n_head=n_head,
            d_word_vec=d_word_vec, d_model=d_model,
            d_inner_hid=d_inner_hid, dropout=dropout)
        self.tgt_word_proj = nn.Linear(d_model, n_tgt_vocab, bias=False)
        nn.init.xavier_normal_(self.tgt_word_proj.weight)
        self.dropout = nn.Dropout(dropout)

        assert d_model == d_word_vec  # 各モジュールの出力のサイズは揃える

        if proj_share_weight:
            # 出力言語の単語のEmbeddingと出力の写像で重みを共有する
            assert d_model == d_word_vec
            self.tgt_word_proj.weight = self.decoder.tgt_word_emb.weight

    def get_trainable_parameters(self):
        # Positional Encoding以外のパラメータを更新する
        enc_freezed_param_ids = set(map(id, self.encoder.position_enc.parameters()))
        dec_freezed_param_ids = set(map(id, self.decoder.position_enc.parameters()))
        freezed_param_ids = enc_freezed_param_ids | dec_freezed_param_ids
        return (p for p in self.parameters() if id(p) not in freezed_param_ids)

    def forward(self, src, tgt):
        src_seq, src_pos = src
        tgt_seq, tgt_pos = tgt

        src_seq = src_seq[:, 1:]
        src_pos = src_pos[:, 1:]
        tgt_seq = tgt_seq[:, :-1]
        tgt_pos = tgt_pos[:, :-1]

        enc_output, *_ = self.encoder(src_seq, src_pos)
        dec_output, *_ = self.decoder(tgt_seq, tgt_pos, src_seq, enc_output)
        seq_logit = self.tgt_word_proj(dec_output)

        return seq_logit

+ Transformerクラス
  + EncoderとDecoderを順次処理
  + 出力に対し線形処理を実行

【参考文献】
1. 斎藤康毅著『セロから作るDeepLearning2 自然言語処理編』オライリー・ジャパン 2018.07
2. 巣籠悠輔著『詳解ディープラーニング 第2版』マイナビ出版 2019.11

## Section6: 物体検知・セグメンテーション
+ 物体認識  
  入力は画像（カラー・白黒は問わず）
  + 分類：画像に対してクラスラベルが出力(1or多)⇒物体の有無
  + 物体検知：バウンディングボックスが出力⇒物体の位置
  + 意味領域分割：各ピクセルにカテゴリラベルの出力
  + 個体領域分割：各ピクセルに個体別ラベルの出力⇒個々の物体の位置
+ Object Detection
  + 入力：画像
  + 出力：バウンディングボックス，ラベル，確率(信頼度)
+ 代表的なデータセット

||ラベル|クラス|Train+Val|Box数|備考|
|---|:---:|---:|---:|---:|---|
|VOC12|○|20|11,540|2.4|現在は終了|
|ILSVRC17||200|476,668|1.1|ImageNetのサブセット．コンペは終了|
|MS COCO18|○|80|123,287|7.3|マイクロソフト|
|OICOD18|○|500|1,743,042|7.0|OpenImagesV4のサブセット|

  + 目的に応じてデータセットを選択
    + Box数
    + クラス数が多いのが正義？

+ 評価指標
  + Confution Matrix
    + precision: 正と予測したものがどれだけ正しいか
    + recall: 実際に正であるものをどれだけ予測できたか
  + Precision-Recall Curve
    + confidenceの閾値を変化させてグラフ化
  + 閾値変化による物体検知
    + 閾値が変化してもクラス分類ではサンプル数は変わらない
    + 物体検知では検知される物体の数が変わる
  + 物体位置の予測
    + IoU: Intersection over Union(Jaccard係数)
$$
IoU = \frac{TP}{TP+FP+FN}=\frac{\text{Area of Overlap}}{\text{Area of Union}}
$$
    + 同じ物体を複数回検出した場合はconfとIoUが閾値を超えた中で最も数値の高いものをTPとなり，それ以外はFPとなる
    + 検出できない物体はFNとなる
    + AP: Avrage Precision
      + IoUの値を固定したときのPRcueveをp(r)とすると
      $$
      AP = \int_0^1 p(r) dr
      $$
    + mAP: mean Average Precision  
    全てのクラスに対するAP平均
    $$
    mAP = \frac{1}{C} \sum_{i=1}^{C} AP_i
    $$
    + IoUも0.5から0.05刻みで計算(MS COCO)
    $$
    mAP_{coco} = \frac{1}{10} \sum_{j=0.5}^{0.95}\frac{1}{C} \sum_{i=1}^{C} AP_{j,i}
    $$
    + 検出速度
      + FPS: Frames per Second：1秒当たりの処理枚数
    + 速度と精度の関係性も検討
+ 深層学習以降の物体検知
  + 2段階検出器：候補領域とクラス推定を別々に行う  
  候補領域の特定⇒画像切り出し⇒クラス推定  
  RCNN, SPPNet, Fast RCNN, Faster RCNN, RFCN, FPN, Mask RCNN
    + 精度が高い
    + 計算量が大きく推論も遅い

  + 1段階検出器：候補領域とクラス推定を同時に行う  
  DetectorNet, SSD, YOLO, YOLO9000, RetinaNet, CornerNet
    + 精度が低い
    + 計算量が小さく推論も速い

+ SSD: Single Shot Detector
  + デフォルトボックスを設定
    + 1つの特徴量に複数のデフォルトボックスを用意
    + k個のデフォルトボックスを用意するとk×(クラス数＋オフセット4)×マップサイズm×n
  + 処理を繰り返し検出したい物体に当てはまるボックスを見つける
+ SSD300
  + input:(3,300,300)
  + conv4_3:(512,38,38) デフォルトボックス: 4×(21＋4)×38×38 = 5776×(21+4)
  + conv7:(1024,19,19) デフォルトボックス: 6×(21＋4)×19×19 = 2166×(21+4)
  + conv8_2:(512,10,10) デフォルトボックス: 6×(21＋4)×10×10 = 600×(21+4)
  + conv9_2:(256,5,5) デフォルトボックス: 6×(21＋4)×5×5 = 150×(21+4)
  + conv10_2:(256,3,3) デフォルトボックス: 4×(21＋4)×3×3 = 36×(21+4)
  + conv11_2:(256,1,1) デフォルトボックス: 4×(21＋4)×1×1 = 4×(21+4)
+ 特徴マップの解像度が高いと小さいものを検知しやすい
+ 複数のデフォルトボックスへの対応
  + Non-Maximum Suppression: 閾値以上で最大の信頼度のものに絞る
  + Hard Negative Mining: 背景部分の学習に制限をつける(物体の3倍まで）
+ 損失関数
  + 信頼度と位置の2項目に関する損失を計算する

+ Semantic Segmentation(DeconvNet・SegNet)
  + 畳み込みとプーリングで解像度が落ちた状態からどのようにUP-Samplingするか  
  ⇒Deconvolution(転置畳み込み)

    + 特徴マップの間に隙間を空ける
    + 特徴マップの周りに余白を入れる
    + 畳み込み演算を行う
    + 輪郭情報の補完  
  解像度が落ちた状態から元の解像度に戻すときにローカル(輪郭)情報が欠落している
    + 途中段階の出力を組み合わせて情報を補完
    + FCN(Fully Convolutional Network): Up-samplingと低レイヤの要素ごとに加算
    + U-Net: Up-samplingと低レイヤチャネル方向で結合  

    ⇒Unpooling

    + プーリング時前の位置を保持
  + Dilated Convolution(拡張畳み込み)
    + 畳み込み演算の段階で受容野を広げられる

【参考文献】
1. チーム・カルポ著『物体検出とＧＡＮ、オートエンコーダー、画像処理入門 』秀和システム 2021.08

## DCGAN
+ GAN(Generative Adversarial Nets)
  + Generator: 乱数からデータを生成
  + Discriminator: 入力データが学習データか生成データかを識別
  + 2プレイヤーミニマックスゲーム
    + 生成器は識別器に学習データである判断させようとトレーニング
    + 識別器は正しく識別できるようにトレーニング
$$
    \underset{G}{min}~ \underset{D}{max} V(D, G)\\
    V(D, G) = \mathbb{E}_{x\sim p_{data(x)}}[\log D(x)]+\mathbb{E}_{z\sim p_{z(z)}}[1 - \log D(G(z))]
$$
  + 価値観数の最適化
    + 価値観数はバイナリクロスエントロピーと同等
    + 識別器は生成器のパラメータを固定して勾配上昇法を使う(max)
      + 学習には学習データと生成データを利用して複数回更新
      + 識別器はノイズと学習データから価値観数を演算
    + 生成器は識別器のパラメータを固定して勾配降下法を使う(min)
      + 学習には生成データを利用して1回更新
      + 生成器にはノイズから生成データを作る
    + 最適化されると生成の確率分布と学習データの確率分布が等しくなるはず  

      ここでx=G(z)とすると，価値関数を最大化するD(x)は
$$
D(x) = \frac{p_{data(x)}}{p_{data(x)} + p_{g(x)}}
$$
これを価値関数の式に代入すると
$$
V = \mathbb{E}_{x\sim p_{data(x)}} \left [ \log\frac{p_{data(x)}}{p_{data(x)} + p_{g(x)}} \right ] +\mathbb{E}_{z\sim p_{z(z)}}\left [1 - \log \frac{p_{g(x)}}{p_{data(x)} + p_{g(x)}} \right ]
= 2JS(p_{data} || ~p_{g}) -2 \log2
$$
JSダイバージェンスは非負で2つの確率分布が等しいときに0となり，このときが最小

+ DCGAN(Deep Convolutional GAN)
  + 畳み込み深層学習を使った画像生成モデル
  + 学習の安定化策
    + プーリング層は使わず，識別器にはストライド2の畳み込み層，生成器にはストライド2の転置畳み込み層を使用
    + 生成器の出力層と識別器の入力層以外でバッチ正規化を使用
    + 中間層は畳み込み層のみでのネットワーク構成
    + 生成器の活性化関数は出力層でTanh，それ以外はReLU
    + 識別器の活性化関数は全てLeakyReLU

【参考文献】
1. チーム・カルポ著『物体検出とＧＡＮ、オートエンコーダー、画像処理入門 』秀和システム 2021.08